## <span style="color:purple"> Information extraction: named entities </span>

Named Entity Recognition (NER) is a subtask of information extraction that seeks to locate and classify elements in text into pre-defined categories such as the names of persons, organizations, locations.

EstNLTK has multiple NER models for Estonian. 
The traditional machine learning based model was created by [Tkachenko et al. (2013)](https://aclanthology.org/W13-2412/), and is available through `NerTagger` and `WordLevelNerTagger` components in EstNLTK. This model annotates names from categories `PER`, `LOC`, `ORG`.

Neural Bert-based models were created by [Kairit Sirts (2023)](https://openreview.net/pdf?id=4CTnlIc1rhw), and are usable via `EstBERTNERTagger` component in EstNLTK neural. 
There are 2 neural NER models: the first recognises names from the basic categories (`PER`, `LOC`, `ORG`), and the second recognises names from the extended set of categories: `PER`, `LOC`, `ORG`, `GPE`, `MONEY`, `PERCENT`, `PROD`, `TITLE`, `DATE`, `TIME`, `EVENT`.

---

## The basic NER model (traditional ML)

The traditional machine learning based NER is distributed with the EstNLTK main package. 

You can apply NER directly via default resolver:

In [1]:
from estnltk import Text

text = Text('Eesti president on Alar Karis. Eesti Energia on Eesti riigile kuuluv energiaettevõte.')

text.tag_layer('ner')

text.ner

Layer(name='ner', attributes=('nertag',), spans=SL[EnvelopingSpan(['Eesti'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Alar', 'Karis'], [{'nertag': 'PER'}]),
EnvelopingSpan(['Eesti', 'Energia'], [{'nertag': 'ORG'}]),
EnvelopingSpan(['Eesti', 'riigile'], [{'nertag': 'LOC'}])])

You can use `enclosing_text` to obtain exact strings corresponding to named entities:

In [2]:
# Get named entity strings
[named_entity.enclosing_text for named_entity in text.ner]

['Eesti', 'Alar Karis', 'Eesti Energia', 'Eesti riigile']

While NerTagger does not provide lemmatization of names, you can iterate over all words of each named entity, and you can get lemmas for these words from the `morph_analysis` layer:

In [3]:
# Get lemmas of the named entities
for named_entity in text.ner:
    for word in named_entity:
        print(word.text, word.lemma)
    print()

Eesti ['Eesti']

Alar ['Alar']
Karis ['Karis']

Eesti ['Eesti']
Energia ['energia']

Eesti ['Eesti']
riigile ['riik']



Note that lemmas are provided as a list due to ambiguities in the morph analysis layer.

## Usage as a tagger 

Next, we'll see how to use NerTagger as a separate tagger.

Loading the tagger:

In [4]:
from estnltk.taggers import NerTagger

nertagger = NerTagger()

Create a text object and add prerequisite layers

In [5]:
from estnltk import Text

In [6]:
text = Text('''Eesti Vabariik on riik Põhja-Euroopas. Eesti piirneb põhjas üle Soome lahe Soome Vabariigiga.''')
text = text.tag_layer(['sentences', 'morph_analysis'])

Add the NER layer to the text

In [7]:
nertagger.tag(text)

Text(text='Eesti Vabariik on riik Põhja-Euroopas. Eesti piirneb põhjas üle Soome lahe Soome Vabariigiga.')

The nertag attribute shows the category of each named entity, either "LOC" - location, "PER" - person or "ORG" - organization

In [8]:
text.ner

Layer(name='ner', attributes=('nertag',), spans=SL[EnvelopingSpan(['Eesti', 'Vabariik'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Põhja-Euroopas'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Eesti'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Soome', 'lahe'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Soome', 'Vabariigiga'], [{'nertag': 'LOC'}])])

For some use cases it might be better to have the output layer with a tag for each word. This can be used for visualizing or making manual changes to the layer.

In [9]:
from estnltk.taggers import WordLevelNerTagger

word_level_ner = WordLevelNerTagger()

In [10]:
word_level_ner.tag(text)

Text(text='Eesti Vabariik on riik Põhja-Euroopas. Eesti piirneb põhjas üle Soome lahe Soome Vabariigiga.')

Here, the tags are in IOB-format: B- prefix indicates that this token is the beginning of the named entity, I- prefix indicates that this token is inside the named entity. O shows that the token is outside named entities (not part of a named entity).

In [11]:
text.wordner

Layer(name='wordner', attributes=('nertag',), spans=SL[Span('Eesti', [{'nertag': 'B-LOC'}]),
Span('Vabariik', [{'nertag': 'I-LOC'}]),
Span('on', [{'nertag': 'O'}]),
Span('riik', [{'nertag': 'O'}]),
Span('Põhja-Euroopas', [{'nertag': 'B-LOC'}]),
Span('.', [{'nertag': 'O'}]),
Span('Eesti', [{'nertag': 'B-LOC'}]),
Span('piirneb', [{'nertag': 'O'}]),
Span('põhjas', [{'nertag': 'O'}]),
Span('üle', [{'nertag': 'O'}]),
Span('Soome', [{'nertag': 'B-LOC'}]),
Span('lahe', [{'nertag': 'I-LOC'}]),
Span('Soome', [{'nertag': 'B-LOC'}]),
Span('Vabariigiga', [{'nertag': 'I-LOC'}]),
Span('.', [{'nertag': 'O'}])])

---

## Neural NER models

### Local installation

*In order to use the neural NER models, please install [estnltk_neural](https://github.com/estnltk/estnltk/tree/main/estnltk_neural) (v1.7.2+).*

Before applying `EstBERTNERTagger`, you'll need to get the models. Models are not distributed with EstNLTK due to their large size, and need to be downloaded separately:

* If you create a new instance of `EstBERTNERTagger` and the models are missing, you'll be prompted with a question asking for a permission to download the "estbertner" model;
* Alternatively, you can pre-download models manually via `download` function:

```python
from estnltk import download
download("estbertner")
download("estbertner_v2")
```

### Model estbertner

The model [estbertner_v1](https://huggingface.co/tartuNLP/EstBERT_NER) is the default model of `EstBERTNERTagger`. 
It recognizes the following categories of named entities: persons (`PER`), locations (`LOC`) and organizations (`ORG`):

In [12]:
from estnltk_neural.taggers import EstBERTNERTagger
neural_ner = EstBERTNERTagger()
neural_ner

EstBERTNERTagger(input_layers=('words',), output_layers=['estbertner'], output_layers_to_attributes={'estbertner': ['nertag']}, model_location=C:\Programmid\Miniconda3\envs\py38_estnltk_neural\lib\site-packages\estnltk-1.7. ..., type: <class 'str'>, length: 169, nlp=<transformers.pipelines.token_classification.TokenClassificationPipeline object  ..., type: <class 'transformers.pipelines.token_classification.TokenClassificationPipeline'>, tokenizer=BertTokenizer(name_or_path='C:\Programmid\Miniconda3\envs\py38_estnltk_neural\li ..., type: <class 'transformers.models.bert.tokenization_bert.BertTokenizer'>, length: 50004, custom_words_layer=words, batch_size=1750, postfix_expand_suffixes=False, postfix_concat_same_type_entities=False, postfix_remove_infix_matches=False)

In [13]:
from estnltk import Text

text = Text('Eesti president on Alar Karis. Eesti Energia on Eesti riigile kuuluv energiaettevõte.')
# NER requires words layer
text.tag_layer('words')
# Add NER layer
neural_ner.tag(text)

text.estbertner

Layer(name='estbertner', attributes=('nertag',), spans=SL[EnvelopingSpan(['Eesti'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Alar', 'Karis'], [{'nertag': 'PER'}]),
EnvelopingSpan(['Eesti', 'Energia'], [{'nertag': 'ORG'}]),
EnvelopingSpan(['Eesti', 'riigile'], [{'nertag': 'LOC'}])])

### Model estbertner_v2

The model [estbertner_v2](https://huggingface.co/tartuNLP/EstBERT_NER_v2) recognizes named entities of 11 categories: `PER`, `LOC`, `ORG`, `GPE`, `MONEY`, `PERCENT`, `PROD`, `TITLE`, `DATE`, `TIME`, `EVENT`. 
In order to use this model, you first need to download it via EstNLTK's downloader, and then pass it's location to `EstBERTNERTagger` via parameter `model_location`:

In [14]:
# Get the model
from estnltk import download, get_resource_paths
download("estbertner_v2")

Resource 'estbertner_v2_from_tartunlp_hf_2022-12-12' has already been downloaded.


True

In [15]:
# Initialize tagger with the model
model_location = get_resource_paths("estbertner_v2", only_latest=True)
neural_ner2 = EstBERTNERTagger(model_location=model_location, output_layer='estbertner2')

In [16]:
from estnltk import Text

text = Text("Kaia Kanepi (WTA 57.) langes USA-s Charlestonis toimuval WTA 500 kategooria tenniseturniiril konkurentsist "+\
            "kaheksandikfinaalis, kaotades poolatarile Magda Linette'ile (WTA 64.) 3 : 6, 6 : 4, 2 : 6.")
# NER requires words layer
text.tag_layer('words')
# Add NER layer
neural_ner2.tag(text)

text.estbertner2

Layer(name='estbertner2', attributes=('nertag',), spans=SL[EnvelopingSpan(['Kaia', 'Kanepi'], [{'nertag': 'PER'}]),
EnvelopingSpan(['WTA'], [{'nertag': 'ORG'}]),
EnvelopingSpan(['USA-s'], [{'nertag': 'GPE'}]),
EnvelopingSpan(['Charlestonis'], [{'nertag': 'GPE'}]),
EnvelopingSpan(['WTA', '500', 'kategooria'], [{'nertag': 'EVENT'}]),
EnvelopingSpan(['poolatarile'], [{'nertag': 'TITLE'}]),
EnvelopingSpan(['Magda', "Linette'ile"], [{'nertag': 'PER'}]),
EnvelopingSpan(['WTA'], [{'nertag': 'ORG'}])])

### Tokenization of NE layers. Restoring Bert's tokenization

By default, detected named entities will envelop around EstNLTK's words layer. 
However, under the hood, Bert models use a different tokenization and the envelopment around words has been achieved heuristically. 
In many cases, Bert's tokenizer splits a word into multiple tokens (e.g. `'Charlestonis'` will become tokenized into `['Cha', 'rl', 'est', 'onis']`) and Bert's named entity labels may not cover all tokens of a word (e.g. frequently Estonian case endings will not be covered).
The heuristic checks for a minimal (one letter) overlap between a Bert token with a named entity label and a word: if the overlap exists, then the word will obtain corresponding named entity label. 
If there are multiple Bert tokens with named entity labels covering a word, then the word will obtain the label of the first Bert token.

If you want to get the original Bert NE output/tokenization, then use `custom_words_layer=None` to switch off the heuristic. 
As a result, the tagger will create 2 layers: the layer with the original Bert tokenization (`nertokens`), and the NE layer enveloping around the Bert tokenization layer.
Example:

In [17]:
# Switch off mapping NE-s to words layer
neural_ner3 = EstBERTNERTagger(model_location=model_location, output_layer='estbertner3', custom_words_layer=None)

In [18]:
# Add NER layer
neural_ner3.tag(text)

# Examine NER results
text.estbertner3

Layer(name='estbertner3', attributes=('nertag',), spans=SL[EnvelopingSpan(['Kai'], [{'nertag': 'PER'}]),
EnvelopingSpan(['a', 'Kanepi'], [{'nertag': 'PER'}]),
EnvelopingSpan(['W'], [{'nertag': 'ORG'}]),
EnvelopingSpan(['USA'], [{'nertag': 'GPE'}]),
EnvelopingSpan(['Cha', 'rl', 'est', 'onis'], [{'nertag': 'GPE'}]),
EnvelopingSpan(['W', 'TA', '500', 'kategooria'], [{'nertag': 'EVENT'}]),
EnvelopingSpan(['poola'], [{'nertag': 'TITLE'}]),
EnvelopingSpan(['Ma'], [{'nertag': 'PER'}]),
EnvelopingSpan(['gda', 'Line', 'tte', "'", 'ile'], [{'nertag': 'PER'}]),
EnvelopingSpan(['W'], [{'nertag': 'ORG'}])])

In [19]:
text.estbertner3.display()

Kai a Kanepi ( W TA 57.) langes USA -s Charlestonis toimuval WTA 500 kategooria tenniseturniiril konkurentsist kaheksandikfinaalis, kaotades poola tarile Ma gda Linette'ile ( W TA 64.) 3 : 6, 6 : 4, 2 : 6.

In [20]:
# Examine the original Bert's tokenization
text.nertokens

Layer(name='nertokens', attributes=(), spans=SL[Span('Kai', [{}]),
Span('a', [{}]),
Span('Kanepi', [{}]),
Span('(', [{}]),
Span('W', [{}]),
Span('TA', [{}]),
Span('57', [{}]),
Span('.', [{}]),
Span(')', [{}]),
Span('langes', [{}]),
Span('USA', [{}]),
Span('-', [{}]),
Span('s', [{}]),
Span('Cha', [{}]),
Span('rl', [{}]),
Span('est', [{}]),
Span('onis', [{}]),
Span('toimuval', [{}]),
Span('W', [{}]),
Span('TA', [{}]),
Span('500', [{}]),
Span('kategooria', [{}]),
Span('tennise', [{}]),
Span('turniiril', [{}]),
Span('konkurentsist', [{}]),
Span('kaheksandik', [{}]),
Span('finaalis', [{}]),
Span(',', [{}]),
Span('kaotades', [{}]),
Span('poola', [{}]),
Span('tari', [{}]),
Span('le', [{}]),
Span('Ma', [{}]),
Span('gda', [{}]),
Span('Line', [{}]),
Span('tte', [{}]),
Span("'", [{}]),
Span('ile', [{}]),
Span('(', [{}]),
Span('W', [{}]),
Span('TA', [{}]),
Span('64', [{}]),
Span('.', [{}]),
Span(')', [{}]),
Span('3', [{}]),
Span(':', [{}]),
Span('6', [{}]),
Span(',', [{}]),
Span('6', [{}]),
Span(':', [{}]),
Span('4', [{}]),
Span(',', [{}]),
Span('2', [{}]),
Span(':', [{}]),
Span('6', [{}]),
Span('.', [{}])])

---